In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import math
import threading
from nltk.corpus import wordnet
import time
from multiprocessing import Process, Queue
import multiprocessing
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')
import csv 
import requests 
import xml.etree.ElementTree as ET 
import os
import pandas as pd
import re
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from multiprocessing import Pool
import time
import random
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import random
from nltk.stem.porter import *
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from multiprocessing import Pool, Manager
from datetime import datetime

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
total_df = pd.read_pickle("/content/QuestionsDataSet.pkl")

In [ ]:
total_df.head(5)

In [ ]:
for i in range(10):
    print(total_df.iloc[i]['Text'])
    print("="*40)

In [ ]:
print(total_df['Topic'].unique())

In [ ]:
posts_text = total_df['Text'].values
print("Posts_text shape", posts_text.shape)

In [ ]:
posts_text[18]

In [ ]:
import re
#https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
def cleanhtml(raw_html):
    """Remove HTML TAG and convert text to lower case"""
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

In [ ]:
preprocessed_post_text = []
for i in range(posts_text.shape[0]):
    preprocessed_post_text.append(cleanhtml(posts_text[i]))
preprocessed_post_text = np.array(preprocessed_post_text)

In [ ]:
preprocessed_post_text[18]

In [ ]:

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+';
for i in range(preprocessed_post_text.shape[0]):
    preprocessed_post_text[i] = re.sub(url_regex, '', preprocessed_post_text[i]);

In [ ]:
preprocessed_post_text[18]

In [ ]:
def cleanpunc(sentence): 
    """function to clean the word of any punctuation or special characters"""
    cleaned = re.sub(r'[?|!|"|#|:|=|+|_|{|}|[|]|-|$|%|^|&|]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/|-|~|`|>|<|*|$|@|;|→]',r'',cleaned)
    return  cleaned

In [ ]:

for i in range(preprocessed_post_text.shape[0]):
    preprocessed_post_text[i] = cleanpunc(preprocessed_post_text[i])

In [ ]:
preprocessed_post_text[18]

In [ ]:
import re

def decontracted(phrase):
    # https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python same code snippet from assignment as well
    """ expanding and creating common English contractions in text"""
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\n", "", phrase)
    return phrase

In [ ]:
for i in range(preprocessed_post_text.shape[0]):
    preprocessed_post_text[i] = decontracted(preprocessed_post_text[i])

In [ ]:

non_stop_word_removed_posts = preprocessed_post_text

In [ ]:
non_stop_word_removed_posts[18]

In [ ]:

non_stop_word_removed_posts_df = pd.DataFrame(non_stop_word_removed_posts, columns=['non_stopword_removed_preprocessed_text'])
non_stop_word_removed_posts_df.index = total_df.index
non_stop_word_removed_posts_df['Id'] = total_df['Id']
total_df = total_df.merge(non_stop_word_removed_posts_df, on='Id',how='left')
total_df.to_pickle('non_stop_word_removed_posts.pkl')

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text):
    """Removes stop words and alpha numeric values"""
    if type(total_text) is not int:# Numbers doesn't make any sense in searching them
        string = ""
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        return string

In [ ]:
mport time
start_time = time.clock()
# we take each title and we text-preprocess it.
for i in range(preprocessed_post_text.shape[0]):
    preprocessed_post_text[i] = nlp_preprocessing(preprocessed_post_text[i], i)
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

In [ ]:
preprocessed_post_text[18]

In [ ]:
preprocesses_text_df = pd.DataFrame(preprocessed_post_text, columns=['preprocessed_text'])
preprocesses_text_df.index = total_df.index
preprocesses_text_df['Id'] = total_df['Id']
total_df = total_df.merge(preprocesses_text_df, on='Id',how='left')

In [ ]:
print("Shape", total_df.shape)
total_df.head(5)

In [ ]:
total_df = total_df[total_df['preprocessed_text'] != '']
total_df = total_df[total_df['preprocessed_text'] != ' ']
total_df = total_df.reset_index(drop=True)
total_df.to_pickle('Preprocessed_questions_text.pkl')